In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow import keras

TensorFlow version: 2.9.1


In [2]:
import os
import sys
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import csv
import pickle
from scipy.stats import linregress
from sklearn.model_selection import train_test_split
from datetime import datetime
import numpy as np

## Loading a dataset

## Split into train data and test data as dataset
- データセットとして使用するデータを取得する。
- trainデータとtestデータに分割する。（例えば80%をtrainデータに、20%をtestデータに、など。）
- 書き方はデータセットとして使用するデータの type による。もし、numpy の形で持っているならhttps://www.tensorflow.org/tutorials/load_data/numpy?hl=ja が参考になります。

In [36]:
x_fullPath = os.path.abspath('../../data/npy/x_l1_1.npy')
x_path = tf.keras.utils.get_file('x_l1_1.npy', 'file://'+x_fullPath)
x_data = np.load(x_path)

y_fullPath = os.path.abspath('../../data/npy/y_l1_1.npy')
y_path = tf.keras.utils.get_file('y_l1_1.npy', 'file://'+y_fullPath)
print(y_path)
y_data = np.load(y_path, allow_pickle=True)

print(y_data)
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2)


# print(x_train.shape)
# print(x_test.shape)
# print(y_train.shape)
# print(y_test.shape)

# print(len(x_train), 'train examples')
# print(len(x_test), 'test examples')
# print(len(y_train), 'train examples')


[{'a a': 0, 'a a*': 0, 'a b': 1, 'a b*': 1, 'a* a': 1, 'a* a*': 0, 'a* b': 1, 'a* b*': 1, 'b a': 0, 'b a*': 0, 'b b': 0, 'b b*': 0, 'b* a': 0, 'b* a*': 0, 'b* b': 0, 'b* b*': 0}
 {'a a': 0, 'a a*': 0, 'a b': 1, 'a b*': 1, 'a* a': 1, 'a* a*': 0, 'a* b': 1, 'a* b*': 1, 'b a': 0, 'b a*': 0, 'b b': 0, 'b b*': 0, 'b* a': 0, 'b* a*': 0, 'b* b': 0, 'b* b*': 0}
 {'a a': 0, 'a a*': 0, 'a b': 1, 'a b*': 1, 'a* a': 1, 'a* a*': 0, 'a* b': 1, 'a* b*': 1, 'b a': 0, 'b a*': 0, 'b b': 0, 'b b*': 0, 'b* a': 0, 'b* a*': 0, 'b* b': 0, 'b* b*': 0}
 {'a a': 0, 'a a*': 0, 'a b': 1, 'a b*': 1, 'a* a': 1, 'a* a*': 0, 'a* b': 1, 'a* b*': 1, 'b a': 0, 'b a*': 0, 'b b': 0, 'b b*': 0, 'b* a': 0, 'b* a*': 0, 'b* b': 0, 'b* b*': 0}
 {'a a': 0, 'a a*': 0, 'a b': 1, 'a b*': 1, 'a* a': 1, 'a* a*': 0, 'a* b': 1, 'a* b*': 1, 'b a': 0, 'b a*': 0, 'b b': 0, 'b b*': 0, 'b* a': 0, 'b* a*': 0, 'b* b': 0, 'b* b*': 0}
 {'a a': 0, 'a a*': 0, 'a b': 1, 'a b*': 1, 'a* a': 1, 'a* a*': 0, 'a* b': 1, 'a* b*': 1, 'b a': 0, 'b a*': 0,

## Set my model
- input_shape は入力データのサイズ。
- モデルの定義方法は調べればたくさん出てきますが、例えばここ： https://sinyblog.com/deaplearning/keras_how_to/ 
- 最後の層の出力は、期待する出力データのサイズ
- [model](https://github.com/si-tm/docs-l10n/blob/master/site/ja/tutorials/quickstart/beginner.py)

In [19]:
def build_model():
    
    model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(x_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'), # units=128 : 出力空間の次元数
    tf.keras.layers.Dropout(0.2), # 入力にドロップアウトを適用する rate=0.2 : 入力ユニットをドロップする割合
    tf.keras.layers.Dense(1) 
    ])

    optimizer = tf.keras.optimizers.Adam() # optimizers も Adam 以外に色々種類があります。調べてみてください！

    model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse']) # loss 関数に何を採用するかはどんな問題を解きたいのかによります。
                                        #ここでは MSE を採用していますが、他にも色々な選択肢があります。調べてみてください！
    return model

In [12]:
model = build_model()

2023-01-17 15:23:06.383729: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 16)                0         
                                                                 
 dense (Dense)               (None, 128)               2176      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 2,305
Trainable params: 2,305
Non-trainable params: 0
_________________________________________________________________


In [14]:
example_batch = x_train[:10]
print(example_batch.shape)
example_result = model.predict(example_batch)
# example_result

(10, 16)
1/1 [==============================] - 1s 799ms/step


In [15]:
print(example_result)

[[-0.32685703]
 [-0.32685703]
 [-0.11339219]
 [-0.10880524]
 [-0.11339219]
 [-0.10880524]
 [-0.12543933]
 [-0.11339219]
 [-0.10880524]
 [-0.32685703]]


In [17]:
# エポックが終わるごとにドットを一つ出力することで進捗を表示
class PrintDot(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: print('')
        print('.', end='')

# early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
# EPOCHS = 400 # epoch 数も考慮しよう
EPOCHS = 100 # epoch 数も考慮しよう

history = model.fit(
    x_train, y_train,
    epochs=EPOCHS, validation_split = 0.2, verbose=0,
    callbacks=[PrintDot()]
)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type dict).

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error [mc]')
    plt.plot(hist['epoch'], hist['mae'], label='Train Error')
    plt.plot(hist['epoch'], hist['val_mae'], label = 'Val Error')
    plt.ylim([0,30])
    plt.legend()

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error [$mc^2$]')
    plt.plot(hist['epoch'], hist['mse'], label='Train Error')
    plt.plot(hist['epoch'], hist['val_mse'], label = 'Val Error')
    plt.ylim([0,400])
    plt.legend()
    plt.show()


plot_history(history)

In [ ]:
test_predictions = model.predict(x_test).flatten()
res = linregress(test_predictions, y_test)

plt.figure(figsize=(6,6))
plt.scatter(y_test, test_predictions)
plt.xlabel('True Values [mc]')
plt.ylabel('Predictions [mc]')
plt.axis('equal')
plt.axis('square')
plt.xlim([-1,100])
plt.ylim([-1,100])
_ = plt.plot([-100, 100], [-100, 100])

plt.plot([-1, 100], res.intercept + res.slope*np.array([-1, 100]), 'r', label='fitted line')

In [ ]:
# トレーニングした model をどこかで使用したいなら以下のようにモデルを保存する
model.save('saved_model/my_model')